# Language Modeling

Generate text using a trigram language model.

Go to https://drive.google.com/drive/folders/1JqAnRSkJqAWlHQRR8tN9is3vKZ-4VKWM?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>


Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


Let's load the trigrams first.

In [ ]:
from math import log

bigram_prefix_to_trigram = {}
bigram_prefix_to_trigram_weights = {}

lines = open("/content/drive/My Drive/nl2ds/tweets/covid-tweets-2020-08-10-2020-08-21.trigrams.txt").readlines()
for line in lines:
  word1, word2, word3, count = line.strip().split()
  if (word1, word2) not in bigram_prefix_to_trigram:
    bigram_prefix_to_trigram[(word1, word2)] = []
    bigram_prefix_to_trigram_weights[(word1, word2)] = []
  bigram_prefix_to_trigram[(word1, word2)].append(word3)
  bigram_prefix_to_trigram_weights[(word1, word2)].append(int(count))

# freeup memory
lines = None

### 1: Retrieving top next words

Retrieving the top next words and their probability given a bigram prefix.

For the following prefixes **word1=middle, word2=of, and n=10**, the output is:

In [ ]:
def top_next_word(word1, word2, n=10):
  next_words = []
  probs = []
  a = bigram_prefix_to_trigram[(word1, word2)]
  s = sum(bigram_prefix_to_trigram_weights[(word1, word2)])
  if len(a) < n:
    n = len(a)
  for i in range(n):
    next_words.append(bigram_prefix_to_trigram[(word1, word2)][i])
    probs.append(bigram_prefix_to_trigram_weights[(word1, word2)][i] / s)
  return next_words, probs

next_words, probs = top_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
covid 0.009389671361502348
nowhere 0.008450704225352112
it 0.004694835680751174
lockdown 0.002347417840375587
summer 0.002347417840375587


## 2: Sampling n words

Sampling the next n words given a bigram prefix without repetition using the probablity distribution defined by the frequency counts.


For the following prefixes **word1=middle, word2=of, and n=10**, the output could be as follows:



In [ ]:
import numpy as np
def sample_next_word(word1, word2, n=10):
  a = bigram_prefix_to_trigram[(word1, word2)]
  weights = bigram_prefix_to_trigram_weights[(word1, word2)]
  probs = []
  next_probs = []
  s = sum(weights)
  l = len(a)
  for i in range(l):
    probs.append(weights[i] / s)  
  if l < n:
    n = l
  next_words = np.random.choice(a, size=n, replace=False, p=probs)
  for i in next_words:
    next_probs.append(probs[a.index(i)])
  next_words = next_words.tolist()
  return next_words, next_probs

next_words, probs = sample_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

a 0.807981220657277
this 0.016901408450704224
the 0.06948356807511737
pandemic 0.023943661971830985
covid 0.009389671361502348
nowhere 0.008450704225352112
them 0.00046948356807511736
an 0.0107981220657277
summer 0.002347417840375587
<EOS> 0.0009389671361502347


## 3: Generate sentences starting with a prefix

Generates n-sentences starting with a given sentence prefix. Uses [beam search](https://en.wikipedia.org/wiki/Beam_search) to generate multiple sentences.

Methods are:

* `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`

* `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`

* `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`

* `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`

In [ ]:
import sys
import heapq

def generate_sentences(prefix, beam, sampler):
  # class for Beam search using heap queue
  class BeamTree(object):
    def __init__(self, width):
        self.heap = list()
        self.width = width
    def add(self, prob, eos, prefix, sentence):
        heapq.heappush(self.heap, (prob, eos, prefix, sentence))
        if len(self.heap) > self.width:
            heapq.heappop(self.heap)
    def get(self):
        return self.heap
    def __iter__(self):
        return iter(self.heap)

  prevB = BeamTree(beam)
  terms = prefix.split()
  # initialize heapq
  prevB.add(1.0, False, [ terms[1], terms[2] ] , [ terms[0], terms[1], terms[2] ])
  while True:
    newB = BeamTree(beam)
    for (prefix_prob, eos, prefixN, sentence) in prevB:
      if eos == True: # if reached EOS, sentence finished
          newB.add(prefix_prob, True, prefixN, sentence)
      else:
          # get next_words and next_probs from the sampler function
          next_words, next_probs = sampler(prefixN[0], prefixN[1], 10)
          for i in next_words:
            if i == '<EOS>':
              newB.add(prefix_prob*next_probs[next_words.index(i)], True, prefixN, sentence + [i])
            else:
              newB.add(prefix_prob*next_probs[next_words.index(i)], False, [prefixN[1]] + [i], sentence + [i])
    
    # check if all sentences are finished
    isFinished = [i[1] for i in newB.get()]
    if all(isFinished) == True: # if all sentences are finished, return the results
      res3 = sorted([(i[3], i[0]) for i in newB.get()], key = lambda x: x[1], reverse=True)
      res1, res2 = zip(*res3)
      res1 = [' '.join(x) for x in res1]
      return res1, res2

    prevB = newB
    
sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=10, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)

<BOS1> <BOS2> trump eyes new unproven coronavirus treatment URL <EOS> 0.00021893147502903603
<BOS1> <BOS2> trump eyes new unproven coronavirus cure URL <EOS> 0.0001719607222046247
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL <EOS> 9.773272077557522e-05
<BOS1> <BOS2> trump eyes new unproven coronavirus therapeutic mypillow creator over unproven therapeutic URL <EOS> 8.212549111137046e-05
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL via @USER <EOS> 7.432226908194607e-06
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven and dangerous <EOS> 5.61685494684627e-06
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven and dangerous covid-19 treatment URL <EOS> 5.235550241426875e-06
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by ben carson and mypillow founder and ceo of mypillow URL <EOS> 2.14